# Cross Directional Movement Indicator

In [2]:
%load_ext memory_profiler

In [3]:
import pandas as pd 
import numpy as np
import time
%mem np.zeros(100000)
%time np.zeros(100000)

peak memory: 98.70 MiB, increment: 0.96 MiB
12.9 µs ± 9.06 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [24]:
df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
print(f"Dataframe shape:{df.shape}")
df = df.drop(columns=['ts','period','currency_pair','volume','quoteVolume','quoteVolume','weightedAverage'])
df.head()

Dataframe shape:(2192, 11)


,date,high,low,open,close
0,1425168000,251.00,240.000121,240.000121,251.00000
1,1425254400,269.00,251.000000,252.000000,269.00000
2,1425340800,274.50,255.000000,268.000000,267.22000
3,1425427200,280.00,258.875000,258.875000,280.00000
4,1425513600,280.89,257.000000,257.000000,257.00012


In [6]:
def true_range(df):
    ranges = [df.high - df.low,(df.high - df.close.shift(1)).abs(),(df.low - df.close.shift(1)).abs()]
    return pd.Series(np.maximum.reduce(ranges))

In [7]:
def directional_movement(df):
    delta_high = df.high - df.high.shift(1)
    delta_low = df.low.shift(1) - df.low
    delta_high = np.where(delta_high < 0, 0, delta_high)
    delta_low = np.where(delta_low < 0, 0, delta_low)
    up_dm = np.where(delta_high > delta_low, delta_high, 0)
    down_dm = np.where(delta_low > delta_high, delta_low, 0)
    return pd.Series(up_dm), pd.Series(down_dm)

In [8]:
def average_true_range(tr,n):
    ATR = tr.ewm(com=n-1,adjust=False).mean()
    return ATR

In [9]:
def average_directional_movement_index_v1(df,n):
    
    df['up_dm'], df['down_dm'] = directional_movement(df)
    df['tr'] = true_range(df)   
    
    df['up_DMn'] = df.up_dm.ewm(com=n-1,adjust=False).mean()
    df['down_DMn'] = df.down_dm.ewm(com=n-1,adjust=False).mean()
    df['TRn'] = df.tr.ewm(com=n-1,adjust=False).mean()
    df['up_DI'] = df.up_DMn/df.TRn
    df['down_DI'] = df.down_DMn/df.TRn

    df['DX'] = abs(df.up_DI- df.down_DI)/(df.up_DI + df.down_DI)
    
    df['ADX'] = df.DX.ewm(com=n-1,adjust=False).mean()
    df['ADXR'] = (df.ADX + df.ADX.shift(n-1)) / 2
    
    
   

In [27]:
def cross_directional_movement_indicator(df):
    df['d0'] = df.up_DI - df.down_DI
    df['d1'] = df.up_DI.shift(1) - df.down_DI.shift(1)
    df['d2'] = df.up_DI.shift(2) - df.down_DI.shift(2)
    df['d3'] = df.up_DI.shift(3) - df.down_DI.shift(3)
    df['d4'] = df.up_DI.shift(4) - df.down_DI.shift(4)
    df['cross_up_down'] = np.where((df.d0 > 0 ) & ( df.d1 < 0) & ( df.d2 < df.d1) & ( df.d3 < df.d2) & ( df.d4 < df.d3),1,0)
    df['cross_down_up'] = np.where((df.d0 < 0 ) & ( df.d1 > 0) & ( df.d2 > df.d1) & ( df.d3 > df.d2) & ( df.d4 > df.d3),1,0)
    

In [37]:
average_directional_movement_index_v1(df,14)
%time cross_directional_movement_indicator(df)
df = df[100:].copy()
df.tail()
print(f"ADX max {df.ADX.max()}")
print(f"ADX min {df.ADX.min()}")
#df.to_csv("output.csv")

CPU times: user 3.98 ms, sys: 0 ns, total: 3.98 ms
Wall time: 3.84 ms
ADX max 0.5991445049143201
ADX min 0.08809580975504952
